<a href="https://colab.research.google.com/github/pdrobny/MonReader/blob/main/tts_kokoro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q kokoro>=0.9.2 soundfile
!apt-get -qq -y install espeak-ng > /dev/null 2>&1

In [ ]:
import IPython.display as ipd
from google.colab import userdata
import numpy as np
import pandas as pd
import os
import PIL.Image
import torch
import torchaudio as ta
import textwrap
from kokoro import KPipeline
from IPython.display import display, Audio
from pydub import AudioSegment
import soundfile as sf



os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import warnings
warnings.filterwarnings('ignore')
print("Imported libraries.")

Imported libraries.


In [ ]:
# load files from google drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Example text

In [ ]:
pipeline = KPipeline(lang_code='a')
text = '''
[Kokoro](/kˈOkəɹO/) is an open-weight TTS model with 82 million parameters. Despite its lightweight architecture, it delivers comparable quality to larger models while being significantly faster and more cost-efficient. With Apache-licensed weights, [Kokoro](/kˈOkəɹO/) can be deployed anywhere from production environments to personal projects.
'''
generator = pipeline(text, voice='af_heart')
for i, (gs, ps, audio) in enumerate(generator):
    print(i, gs, ps)
    display(Audio(data=audio, rate=24000, autoplay=i==0))
    # Save the audio using soundfile
    sf.write(f'{i}.wav', audio.numpy(), 24000)

## Extracted Text

In [ ]:
# Read text from the file
text_file_path = "/content/drive/MyDrive/Apziva/MonReader/extract/p3_extract.txt" # Assuming the next extracted text is in p4_extract.txt
try:
    with open(text_file_path, 'r') as f:
        book_page_text = f.read().strip()
except FileNotFoundError:
    print(f"Error: Text file not found at {text_file_path}")
    book_page_text = "" # Set empty string if file not found
cleaned_text = book_page_text.replace('\n', ' ')
cleaned_text

"IDEAS ABOUT THE UNIVERSE 3  As long ago as 340 B.C. Aristotle, in his book On the Heavens, was able to put forward two good arguments for believ- ing that the Earth was a round ball rather than a flat plate. First, he realized that eclipses of the moon were caused by the Earth coming between the sun and the moon. The Earth's shadow on the moon was always round, which would be true only if the Earth was spherical. If the Earth had been a flat disk, the shadow would have been elongated and elliptical, unless the eclipse always occurred at a time when the sun was di- rectly above the center of the disk. Second, the Greeks knew from their travels that the Pole Star appeared lower in the sky when viewed in the south than it did in more northerly regions. From the difference in the ap- parent position of the Pole Star in Egypt and Greece, Aristotle even quoted an estimate that the distance around the Earth was four hundred thousand stadia. It is not known exactly what length a stadium was, 

In [ ]:
pipeline = KPipeline(lang_code='a') # Changed language code to 'a' for American English

In [ ]:
def chunk_text(text, max_chars=200):
    return textwrap.wrap(text, width=max_chars)

In [ ]:
# generate audio for each chunk
chunks = chunk_text(cleaned_text)
os.makedirs("segments", exist_ok=True)
segment_files = []

for i, chunk in enumerate(chunks):
    # Use the pipeline to generate audio for the chunk
    generator = pipeline(chunk, voice='af_heart')
    # The pipeline yields (gs, ps, audio), we need the audio tensor
    for j, (gs, ps, audio) in enumerate(generator):
        file_name = f"segments/segment_{i}_{j}.wav"
        # Save the audio using soundfile, using the known sample rate
        sf.write(file_name, audio.numpy(), 24000)
        segment_files.append(file_name)

In [ ]:
# stitch together audio
final_audio = AudioSegment.empty()
for file in segment_files:
    seg = AudioSegment.from_wav(file)
    final_audio += seg

final_audio.export("final_output.wav", format="wav")

<_io.BufferedRandom name='final_output.wav'>

In [ ]:
audio_data, sample_rate = ta.load("final_output.wav")
ipd.Audio(audio_data, autoplay=True, rate=sample_rate)